In [1]:
import pytest
import tensorflow as tf
import numpy as np
from upstride import generic_layers

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [2]:
l = generic_layers.Dense(1, ['', '12'], (2, 0, 0), units=1, kernel_initializer='ones')
img = np.ones((2, 1), dtype='float32')
out = l(img)
ref = [[0], [2]]
assert np.all(out == ref)

In [3]:
def assert_small_float_difference(tensor1, tensor2, relative_error_threshold):
    """ Asserts float tensors differ by no more than threshold scaled by the values checked
    """
    abs_diff = tf.abs(tensor1 - tensor2)
    abs_max_tensors = tf.abs(tf.maximum(tensor1, tensor2))
    threshold = relative_error_threshold * (1 + abs_max_tensors)
    assert tf.reduce_all(abs_diff < threshold)

In [4]:
def assert_small_float_difference_numpy(tensor1, tensor2, relative_error_threshold):
    """ Asserts float tensors differ by no more than threshold scaled by the values checked
    """
    abs_diff = np.abs(tensor1 - tensor2)
    abs_max_tensors = np.abs(np.maximum(tensor1, tensor2))
    threshold = relative_error_threshold * (1 + abs_max_tensors)
    assert (abs_diff < threshold).all()

In [ ]:
# def test_dense_simplest():
#     l = generic_layers.Dense(1, ['', '12'], (2, 0, 0), units=1, kernel_initializer='ones')
#     img = np.ones((2, 1))
#     out = l(img)
#     ref = [[0], [2]]
#     assert np.all(out == ref)

def test_dense():
    l = generic_layers.Dense(1, ['', '12'], (2, 0, 0), units=1, kernel_initializer='ones')
    tensor_len = 2
    real = np.random.rand(tensor_len)
    imaginary = np.random.rand(tensor_len)
    img = np.concatenate((real, imaginary))
    img = np.expand_dims(img, axis=1)
    comp = real + imaginary * 1j
    out = l(img)

    print(img)
    print(comp)
    print(out)

In [5]:
l = generic_layers.Dense(1, ['', '12'], (2, 0, 0), units=1)
tensor_len = 3
real = np.random.rand(tensor_len)
imaginary = np.random.rand(tensor_len)
img = np.concatenate((real, imaginary))
img = np.expand_dims(img, axis=1).astype('f')
out = l(img)
real_out, imaginary_out = np.split(out, 2, axis=0)
fin = real_out + imaginary_out * 1j

In [6]:
comp = real + imaginary * 1j
w = l.get_weights()[0]
w_comp = w[0][0] + w[0][1] * 1j
out_comp = comp * w_comp
fin_2 = np.expand_dims(out_comp, axis=1)

In [7]:
assert_small_float_difference_numpy(fin, fin_2, 0.00001)

In [8]:
l = generic_layers.Dense(1, ['', '12'], (2, 0, 0), units=1)
tensor_len = 3
real = np.random.rand(tensor_len)
imaginary = np.random.rand(tensor_len)
img = np.concatenate((real, imaginary))
img = np.reshape(img, (2, tensor_len)).astype('f')
out = l(img)
out = np.squeeze(out)

In [9]:
comp = real + imaginary * 1j
w = l.get_weights()[0]
w = np.array(list(map(lambda x : x[0] + x[1] * 1j, w)))

In [10]:
comp_out = sum(w * comp)
comp_out = [comp_out.real, comp_out.imag]

In [11]:
assert_small_float_difference_numpy(out, comp_out, 0.00001)

In [12]:
l = generic_layers.Dense(1, ['', '12'], (2, 0, 0), units=1, kernel_initializer='ones')

In [13]:
img = np.ones((8, 4), dtype='float32')
out = l(img)

In [14]:
images = 2
channels = 3
size = images * channels
real = np.random.rand(size)
imag = np.random.rand(size)
inp = np.concatenate((real, imag)).reshape(images * 2, channels).astype('f')

In [15]:
inp

array([[0.2301261 , 0.11060228, 0.97221565],
       [0.1073442 , 0.31989068, 0.6991877 ],
       [0.8913892 , 0.9082932 , 0.72102904],
       [0.5737233 , 0.02749028, 0.20947774]], dtype=float32)

In [16]:
uni = 1 + 1j

In [17]:
comp = real + imag * 1j

In [18]:
interm = comp * uni

In [19]:
fin_comp = np.sum(np.split(interm, images), axis=1)

In [20]:
np.concatenate((fin_comp.real, fin_comp.imag))

array([-1.20776734,  0.31573119,  3.83365545,  1.93711389])

In [21]:
l.get_weights()[0]

array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]], dtype=float32)

In [22]:
def test_dense():
    units = 2
    images = 2
    channels = 3
    l = generic_layers.Dense(1, ['', '12'], (2, 0, 0), units=units)
    size = images * channels
    real = np.random.rand(size)
    imag = np.random.rand(size)
    inp = np.concatenate((real, imag)).reshape(images * 2, channels).astype('f')
    out = l(inp).numpy()

    comp = np.array(real + imag * 1j).reshape(images, channels)
    w = l.get_weights()[0]
    real_w, imag_w = np.split(w, 2, axis=1)
    comp_w = real_w + imag_w * 1j
    comp_out = np.sum(np.array([comp * comp_w[:, i] for i in range(units)]), axis=2)
    comp_out = np.transpose(comp_out)
    comp_out = np.concatenate((comp_out.real, comp_out.imag))

    assert_small_float_difference_numpy(out, comp_out, 0.00001)

In [23]:
test_dense()

In [24]:
tf.keras.backend.set_image_data_format('channels_last')

In [25]:
units = 2
images = 2
channels = 3
l = generic_layers.Dense(1, ['', '12'], (2, 0, 0), units=units)
size = images * channels
real = np.random.rand(size)
imag = np.random.rand(size)
inp = np.concatenate((real, imag)).reshape(images * 2, channels).astype('f')
out = l(inp).numpy()

comp = np.array(real + imag * 1j).reshape(images, channels)
w = l.get_weights()[0]
real_w, imag_w = np.split(w, 2, axis=1)
# comp_w = np.array(list(map(lambda x : x[0] + x[1] * 1j, w)))
comp_w = real_w + imag_w * 1j
comp_out = np.sum(np.array([comp * comp_w[:, i] for i in range(units)]), axis=2)
comp_out = np.transpose(comp_out)
# comp_out = np.sum(comp * comp_w, axis=1)
comp_out = np.concatenate((comp_out.real, comp_out.imag))
print(comp_out)
print(out)

assert_small_float_difference_numpy(out, comp_out, 0.00001)

[[-1.01062724  0.8018811 ]
 [-0.57492527 -0.10408887]
 [-0.0246183  -0.26509437]
 [ 0.19089559 -0.96583212]]
[[-1.0106273   0.80188113]
 [-0.57492524 -0.10408884]
 [-0.0246183  -0.26509434]
 [ 0.19089562 -0.9658322 ]]


In [26]:
a = np.transpose(comp_out)
np.concatenate((a.real, a.imag))

array([[-1.01062724, -0.57492527, -0.0246183 ,  0.19089559],
       [ 0.8018811 , -0.10408887, -0.26509437, -0.96583212],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ]])

In [27]:
out

array([[-1.0106273 ,  0.80188113],
       [-0.57492524, -0.10408884],
       [-0.0246183 , -0.26509434],
       [ 0.19089562, -0.9658322 ]], dtype=float32)

In [28]:
l.get_weights()

[array([[-0.27667278, -0.4203325 ,  0.521034  ,  0.60974824],
        [ 0.0855304 , -0.26570588,  0.5654576 , -0.8775375 ],
        [-0.1781258 ,  0.41945064, -0.23306441, -0.4980507 ]],
       dtype=float32),
 array([[0., 0., 0., 0.]], dtype=float32)]

In [29]:
real_w, imag_w = np.split(w, 2, axis=1)

In [30]:
real_w

array([[-0.27667278, -0.4203325 ],
       [ 0.0855304 , -0.26570588],
       [-0.1781258 ,  0.41945064]], dtype=float32)

In [31]:
l = generic_layers.Dense(1, ['', '12'], (2, 0, 0), units=1)
inp = np.ones((2,4), dtype='f')
l(inp)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[ 0.04553461],
       [-2.3930364 ]], dtype=float32)>

In [32]:
kernel_size = 1
batch_size = 1
height = 1
width = 1
channels = 2

l = generic_layers.DepthwiseConv2D(1, ['', '12'], (2, 0, 0), kernel_size=kernel_size, depthwise_initializer='ones')
t_shape = (batch_size, height, width, channels)
# real = np.zeros(t_shape).astype('f') # np.random.randint(3, size=t_shape).astype('f')
# imag = np.ones(t_shape).astype('f') # np.random.randint(3, size=t_shape).astype('f')
real = np.array([1] + [0] * (channels - 1)).reshape(t_shape).astype('f')
imag = np.zeros(t_shape).astype('f')
# imag = np.array([1,0]).reshape(t_shape).astype('f')
inp = np.concatenate((real, imag)).astype('f')
out = l(inp).numpy()

w = l.get_weights()[0]
real_w, imag_w = np.split(w, 2, axis=3)

l_tf = tf.keras.layers.DepthwiseConv2D(kernel_size=kernel_size)
l_tf(real)
l_tf.set_weights([real_w, l_tf.get_weights()[1]])
real_real = l_tf(real)
real_imag = l_tf(imag)
l_tf.set_weights([imag_w, l_tf.get_weights()[1]])
imag_real = l_tf(real)
imag_imag = l_tf(imag)
comp_real = real_real - imag_imag
comp_imag = real_imag + imag_real
comp_out = np.concatenate((comp_real, comp_imag))

assert_small_float_difference_numpy(out, comp_out, 0.00001)

AssertionError: 

In [33]:
tf.keras.backend.set_image_data_format('channels_first')

In [34]:
kernel_size = 1
batch_size = 1
height = 1
width = 1
channels = 2

l = generic_layers.DepthwiseConv2D(1, ['', '12'], (2, 0, 0), kernel_size=kernel_size, depthwise_initializer='ones')
t_shape = (batch_size, channels, height, width)
real = np.array([1] + [0] * (channels - 1)).reshape(t_shape).astype('f')
imag = np.array([3] + [0] * (channels - 1)).reshape(t_shape).astype('f') # np.zeros(t_shape).astype('f')
inp = np.concatenate((real, imag)).astype('f')
out = l(inp).numpy()

w = l.get_weights()[0]
real_w, imag_w = np.split(w, 2, axis=3)

l_tf = tf.keras.layers.DepthwiseConv2D(kernel_size=kernel_size)
l_tf(real)
l_tf.set_weights([real_w, l_tf.get_weights()[1]])
real_real = l_tf(real)
real_imag = l_tf(imag)
l_tf.set_weights([imag_w, l_tf.get_weights()[1]])
imag_real = l_tf(real)
imag_imag = l_tf(imag)
comp_real = real_real - imag_imag
comp_imag = real_imag + imag_real
comp_out = np.concatenate((comp_real, comp_imag))

assert_small_float_difference_numpy(out, comp_out, 0.00001)

AssertionError: 

In [35]:
comp_real

<tf.Tensor: shape=(1, 2, 1, 1), dtype=float32, numpy=
array([[[[-2.]],

        [[ 0.]]]], dtype=float32)>

In [36]:
comp_imag

<tf.Tensor: shape=(1, 2, 1, 1), dtype=float32, numpy=
array([[[[4.]],

        [[0.]]]], dtype=float32)>

In [37]:
real_real

<tf.Tensor: shape=(1, 2, 1, 1), dtype=float32, numpy=
array([[[[1.]],

        [[0.]]]], dtype=float32)>

In [38]:
real_imag

<tf.Tensor: shape=(1, 2, 1, 1), dtype=float32, numpy=
array([[[[3.]],

        [[0.]]]], dtype=float32)>

In [39]:
imag_real

<tf.Tensor: shape=(1, 2, 1, 1), dtype=float32, numpy=
array([[[[1.]],

        [[0.]]]], dtype=float32)>

In [40]:
imag_imag

<tf.Tensor: shape=(1, 2, 1, 1), dtype=float32, numpy=
array([[[[3.]],

        [[0.]]]], dtype=float32)>

In [41]:
inp

array([[[[1.]],

        [[0.]]],


       [[[3.]],

        [[0.]]]], dtype=float32)

In [42]:
w

array([[[[1., 1.],
         [1., 1.]]]], dtype=float32)

In [43]:
w.shape

(1, 1, 2, 2)

In [44]:
inp.shape

(2, 2, 1, 1)

In [45]:
out

array([[[[1.]],

        [[1.]]],


       [[[3.]],

        [[3.]]]], dtype=float32)

In [46]:
comp_out



array([[[[-2.]],

        [[ 0.]]],


       [[[ 4.]],

        [[ 0.]]]], dtype=float32)

In [47]:
out.shape

(2, 2, 1, 1)

In [48]:
real

array([[[[1.]],

        [[0.]]]], dtype=float32)

In [49]:
imag

array([[[[3.]],

        [[0.]]]], dtype=float32)

In [50]:
inp

array([[[[1.]],

        [[0.]]],


       [[[3.]],

        [[0.]]]], dtype=float32)

In [51]:
w

array([[[[1., 1.],
         [1., 1.]]]], dtype=float32)

In [52]:
real_w

array([[[[1.],
         [1.]]]], dtype=float32)

In [53]:
w.shape

(1, 1, 2, 2)

In [54]:
w

array([[[[1., 1.],
         [1., 1.]]]], dtype=float32)

In [ ]:
#     for i in range(hyper_dimension):
#         print(i)
#         partial_out = 0
#         for j in range(hyper_dimension):
#             for k in range(hyper_dimension):
#                 if (j + k) % hyper_dimension == i:
#                     print((j,k))
#                     partial_out = partial_out + ref_partial[j][k] * algebra_map[j][k]
#         ref_outputs.append(partial_out)

In [124]:
filters = 2
kernel_size = 1
batch_size = 2
height = 2
width = 2
channels = 2

l = generic_layers.Conv2D(1, ['', '12'], (2, 0, 0), filters=filters, kernel_size=kernel_size)
t_shape = (batch_size, channels, height, width)
real = np.random.rand(*t_shape).astype('f')
imag = np.random.rand(*t_shape).astype('f')
inp = np.concatenate((real, imag)).astype('f')
out = l(inp).numpy()

w = l.get_weights()[0]
real_w, imag_w = np.split(w, 2, axis=3)

l_tf = tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_size)
l_tf(real)
l_tf.set_weights([real_w, l_tf.get_weights()[1]])
real_real = l_tf(real)
real_imag = l_tf(imag)
l_tf.set_weights([imag_w, l_tf.get_weights()[1]])
imag_real = l_tf(real)
imag_imag = l_tf(imag)
comp_real = real_real - imag_imag
comp_imag = real_imag + imag_real
comp_out = np.concatenate((comp_real, comp_imag))

assert_small_float_difference_numpy(out, comp_out, 0.001)

In [261]:
def test_generic(layer_test, layer_ref, hyper_dimension, algebra_map, tensor_shape):
    assert algebra_map.shape == (hyper_dimension, hyper_dimension, 2)
    
    components = []
    for _ in range(hyper_dimension):
        component = np.random.rand(*tensor_shape).astype('f')
        components.append(component)
    
    inp = np.concatenate(components)
    test_out = layer_test(inp)
    
    w = layer_test.get_weights()[0]
    w_components = np.split(w, hyper_dimension, axis=-1)
    
    layer_ref(components[0])
    bias = layer_ref.get_weights()[1]
    
    ref_partial = [[] for _ in range(hyper_dimension)]
    for i in range(hyper_dimension):
        layer_ref.set_weights([w_components[i], bias])
        for j in range(hyper_dimension):
            inter_res = layer_ref(components[j])
            ref_partial[i].append(inter_res)
    
    ref_outputs = [0 for _ in range(hyper_dimension)]
    for i in range(hyper_dimension):
        for j in range(hyper_dimension):
            which, coeff = algebra_map[i][j]
            ref_outputs[which] = ref_outputs[which] + ref_partial[i][j] * coeff
        
    ref_out = np.concatenate(ref_outputs)
    
#     print(test_out)
#     print(ref_out)
    
    assert_small_float_difference(test_out, ref_out, 0.0001)

In [262]:
layer_test = generic_layers.Conv2D(1, ['', '12'], (2, 0, 0), filters=filters, kernel_size=kernel_size)
layer_ref = tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_size)

In [263]:
test_generic(layer_test, layer_ref, 2, np.array([[(0, 1), (1, 1)], [(1, 1), (0, -1)]]), t_shape)

In [269]:
batch_size = 4
height = 6
width = 10
channels = 8

t_shape = (batch_size, channels, height, width)

In [270]:
layer_kwargs = {
    'filters' : 18,
    'kernel_size' : 2,
}

# uptype = (1, ['', '12'], (2, 0, 0))
# hyper_dimension = 2
# algebra_map = np.array([[(0, 1), (1, 1)], [(1, 1), (0, -1)]])

uptype = (2, ['', '12', '13', '23'], (3, 0, 0))
hyper_dimension = 4
algebra_map = np.array([
    [(0, 1), (1, 1), (2, 1), (3, 1)],
    [(1, 1), (0, -1), (3, 1), (2, -1)],
    [(2, 1), (3, -1), (0, -1), (1, 1)],
    [(3, 1), (2, 1), (1, -1), (0, -1)],
])

layer_test = generic_layers.Conv2D(*uptype, **layer_kwargs)
layer_ref = tf.keras.layers.Conv2D(**layer_kwargs)

In [271]:
test_generic(layer_test, layer_ref, hyper_dimension, algebra_map, t_shape)

In [272]:
layer_kwargs = {
    'units' : 1
}

layer_test = generic_layers.Dense(*uptype, **layer_kwargs)
layer_ref = tf.keras.layers.Dense(**layer_kwargs)

t_shape = (batch_size, channels)

In [273]:
test_generic(layer_test, layer_ref, hyper_dimension, algebra_map, t_shape)